Harmonization Module

> This module encompasses a suite of tools designed for the harmonization of gene expression data derived from different sequencing technologies. Specifically, it provides functionality to standardize short-read gene expression matrices with long-read transcript matrices. The harmonization process ensures that datasets from disparate sources can be directly compared and analyzed together, facilitating integrative studies and enhancing the robustness of biological insights. The tools within this notebook are part of a larger ecosystem aimed at streamlining the analysis of transcriptomic data.


In [1]:
#| default_exp Standardization

In [2]:
#| hide
from nbdev.showdoc import *

Harmonize two datasets by extracting a subset from one dataset containing only the cells that are also present in the other dataset, and by ensuring that the metadata and gene expression measurements are compatible for comparative analysis. This process facilitates the integration of datasets from different sequencing technologies or experimental conditions, allowing for robust downstream analyses such as differential expression, clustering, and trajectory inference.


In [3]:
#| export
import scanpy as sc


def subset_common_cells( dataset1:sc.AnnData,  # First dataset to be subsetted.
                        dataset2:sc.AnnData   # Second dataset to compare with.
                        ) -> sc.AnnData:  # Subset of `dataset1` containing only cells also found in `dataset2`.
    
    "Subset `dataset1` to only include cells that are also present in `dataset2`."
    # Find common cells by intersecting the cell identifiers of both datasets
    common_cells = dataset1.obs_names.intersection(dataset2.obs_names)
    
    # Explicitly subset dataset1 to only include these common cells
    subset_dataset1 = dataset1[common_cells, :].copy()
    
    return subset_dataset1


Transfer the .obs attribute from one AnnData object to another, ensuring compatibility in .obs dimensionality and preserving the .var DataFrame. This function is designed to maintain the integrity of the target dataset's gene expression information while enriching it with the metadata from the source dataset. It is a crucial step in the harmonization process, allowing for integrated analysis of datasets from different sources.


In [4]:
#| export 

import pandas as pd
import anndata as ad

#| export
def transfer_obs(dataset1: ad.AnnData,  # Source AnnData object with .obs metadata to transfer.
                 dataset2: ad.AnnData   # Target AnnData object to receive .obs metadata.
                 ) -> ad.AnnData:       # The modified `dataset2` with .obs from `dataset1` transferred.
    "Transfer `.obs` metadata from `dataset1` to `dataset2` one by one, while preserving the `.var` DataFrame of `dataset2`."
    
    # Ensure dataset2's .var is preserved without altering its contents
    var_dataset2 = dataset2.var.copy()
    
    # Clear current .obs in dataset2 to ensure it only contains metadata from dataset1
    dataset2.obs = pd.DataFrame(index=dataset2.obs.index)
    
    # Loop through each column in dataset1.obs and transfer it to dataset2.obs
    for column_name in dataset1.obs.columns:
        # Transfer each column individually
        dataset2.obs[column_name] = dataset1.obs[column_name].copy()

    # Reapply the saved .var to dataset2 to ensure it's preserved
    dataset2.var = var_dataset2

    return dataset2

In [8]:
#| hide
import nbdev; nbdev.nbdev_export()